In [1]:
import torch.nn
import torch.optim

import openml
import openml_pytorch
import openml_pytorch.layers
import openml_pytorch.config
import logging
from types import SimpleNamespace
import warnings
import pandas as pd

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore')

############################################################################
# Enable logging in order to observe the progress while running the example.
openml.config.logger.setLevel(logging.DEBUG)
openml_pytorch.config.logger.setLevel(logging.DEBUG)
############################################################################

############################################################################
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# openml.config.apikey = 'key'
openml_pytorch.config.file_dir = openml.config.get_cache_directory()+'/datasets/44312/PNU_Micro/images/'
openml_pytorch.config.filename_col = "FILE_NAME"
openml_pytorch.config.perform_validation = False

In [3]:
# Download the OpenML task for the Meta_Album_PNU_Micro dataset.
task = openml.tasks.get_task(361987)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(13456, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 19) # To user - Remember to set correct size of last layer. 

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()

In [5]:
from openml_pytorch.config import \
            criterion_gen, \
            optimizer_gen, scheduler_gen, \
            progress_callback, epoch_count, \
            batch_size, \
            predict, predict_proba, \
            file_dir, filename_col, \
            sanitize, retype_labels, perform_validation, image_size

In [6]:
model_config = SimpleNamespace(
            device='cuda' if torch.cuda.is_available() else 'cpu',
            criterion_gen=criterion_gen,
            optimizer_gen=optimizer_gen,
            scheduler_gen=scheduler_gen,
            perform_validation = perform_validation,
            image_size = image_size
        )

data_config = SimpleNamespace(
    progress_callback=progress_callback,
            epoch_count = epoch_count,
            batch_size = batch_size,
            predict = predict,
            predict_proba = predict_proba,
            file_dir = file_dir,
            filename_col = filename_col,
            sanitize = sanitize,
            retype_labels = retype_labels,

)

In [7]:
openml_pytorch.setup_config(model_config=model_config, data_config=data_config)

In [9]:
run = openml.runs.run_model_on_task(model, task, avoid_duplicate_runs=False)

KeyboardInterrupt: 

In [9]:
run.predictions

,repeat,fold,sample,row_id,prediction,correct,confidence.HeadNeck,confidence.Pancreatic,confidence.Bile-duct,confidence.Thyroid,...,confidence.Cervix,confidence.Testis,confidence.Colon,confidence.Bladder,confidence.Liver,confidence.Stomach,confidence.Kidney,confidence.Skin,confidence.Breast,confidence.Lung
0,0,0,0,11,Kidney,Cervix,0.054072,0.048297,0.053308,0.055061,...,0.047876,0.055564,0.054657,0.048055,0.054201,0.050540,0.058237,0.051373,0.055537,0.054318
1,0,0,0,354,Kidney,Thyroid,0.054072,0.048297,0.053308,0.055061,...,0.047876,0.055564,0.054657,0.048055,0.054201,0.050540,0.058237,0.051373,0.055537,0.054318
2,0,0,0,612,Kidney,Lung,0.054072,0.048297,0.053308,0.055061,...,0.047876,0.055564,0.054657,0.048055,0.054201,0.050540,0.058237,0.051373,0.055537,0.054318
3,0,0,0,222,Kidney,Stomach,0.054072,0.048297,0.053308,0.055061,...,0.047876,0.055564,0.054657,0.048055,0.054201,0.050540,0.058237,0.051373,0.055537,0.054318
4,0,0,0,326,Kidney,Liver,0.054072,0.048297,0.053308,0.055061,...,0.047876,0.055564,0.054657,0.048055,0.054201,0.050540,0.058237,0.051373,0.055537,0.054318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,0,9,0,431,Testis,HeadNeck,0.052045,0.046212,0.054180,0.052565,...,0.052549,0.056122,0.055136,0.047489,0.052369,0.052154,0.050562,0.054220,0.053989,0.055343
756,0,9,0,747,Testis,Pancreatic,0.052045,0.046212,0.054180,0.052565,...,0.052549,0.056122,0.055136,0.047489,0.052369,0.052154,0.050562,0.054220,0.053989,0.055343
757,0,9,0,591,Testis,Skin,0.052045,0.046212,0.054180,0.052565,...,0.052549,0.056122,0.055136,0.047489,0.052369,0.052154,0.050562,0.054220,0.053989,0.055343
758,0,9,0,193,Testis,Breast,0.052045,0.046212,0.054180,0.052565,...,0.052549,0.056122,0.055136,0.047489,0.052369,0.052154,0.050562,0.054220,0.053989,0.055343
